In [122]:
import pandas as pd
import numpy as np

import requests

from datetime import datetime, timedelta
from pprint import pprint

import plotly.express as px
import plotly.colors as pc

### EJEMPLO


In [121]:
lang    =  "es"
category = "balance"
widget   = "balance-electrico"
query   = "start_date=2018-01-01T00:00&end_date=2018-12-31T23:59&time_trunc=year"


In [123]:
headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Host' : 'apidatos.ree.es'
}

In [ ]:
endpoint = f"https://apidatos.ree.es/{lang}/datos/{category}/{widget}?{query}"
response = requests.get(url = endpoint, headers = headers)
data = response.json()
pprint(data)

# OBTENCIÓN DE DATOS DE BALANCE




In [166]:
lang          = 'es'

hoy           = '2023-12-31'
ultima_fecha  = '2023-01-01'

In [163]:
headers = {
    'Accept':       'application/json',
    'Content-Type': 'application/json',
    'Host' :        'apidatos.ree.es'
}

In [167]:
def balance_datos(lang, hoy, ultima_fecha):

    query = f"start_date={ultima_fecha}T00:00&end_date={hoy}T23:59&time_trunc=day"

    endpoint = f"https://apidatos.ree.es/{lang}/datos/balance/balance-electrico?{query}"
    response = requests.get(url = endpoint, headers = headers)
    data = response.json()



    lista_nombres = list()
    lista_tipos = list()
    lista_valores = list()
    lista_porcentajes = list()
    lista_fechas = list()
    lista_dias = list()
    lista_meses = list()
    lista_años = list()


    for dato in data['included']:
        for info in dato['attributes']['content']:
            nombre = info['type']

            tipo = info['groupId']


            for i in info['attributes']['values']:
                valor = i['value']

                porcentaje = i['percentage']

                fecha = i['datetime']
                fecha = pd.to_datetime(fecha)
                dia = fecha.strftime('%d')
                mes = fecha.strftime('%m')
                año = fecha.strftime('%Y')

                fecha = fecha.strftime("%d/%m/%Y")




                lista_nombres.append(nombre)
                lista_tipos.append(tipo)
                lista_valores.append(valor)
                lista_porcentajes.append(porcentaje)
                lista_fechas.append(fecha)
                lista_dias.append(dia)
                lista_meses.append(mes)
                lista_años.append(año)

    df_balance = pd.DataFrame()
    df_balance['nombre']               = lista_nombres
    df_balance['tipo de energía']      = lista_tipos
    df_balance['Valores']              = lista_valores
    df_balance["Porcentaje"]           = lista_porcentajes
    df_balance["Fecha actualización"]  = lista_fechas


    return df_balance

df_balance  = balance_datos(lang, hoy, ultima_fecha)

#df_balance['tipo de energía'].unique()



In [168]:
df_balance

,nombre,tipo de energía,Valores,Porcentaje,Fecha actualización
0,Hidráulica,Renovable,76595.584,0.272156,01/01/2023
1,Hidráulica,Renovable,136518.071,0.491590,02/01/2023
2,Hidráulica,Renovable,153711.167,0.516836,03/01/2023
3,Hidráulica,Renovable,138220.830,0.487514,04/01/2023
4,Hidráulica,Renovable,143184.392,0.538530,05/01/2023
...,...,...,...,...,...
7664,Demanda en b.c.,Demanda en b.c.,696924.348,1.000000,27/12/2023
7665,Demanda en b.c.,Demanda en b.c.,704808.954,1.000000,28/12/2023
7666,Demanda en b.c.,Demanda en b.c.,693457.822,1.000000,29/12/2023
7667,Demanda en b.c.,Demanda en b.c.,624295.946,1.000000,30/12/2023


In [56]:
df_balance.to_csv('balance_electrico.csv')

In [57]:
colores_personalizados = px.colors.qualitative.Plotly + px.colors.qualitative.Pastel + px.colors.qualitative.Set1
colores_personalizados = colores_personalizados[:30]

In [ ]:
fig_all = px.line(data_frame = df_balance,
        x = 'Fecha actualización',
        y = 'Valores',
        color = 'nombre',
        color_discrete_sequence = colores_personalizados,
)

fig_all.update_layout(title = 'Evolución de energía diaría unificada')

fig_all.show()


In [ ]:
fig_reno = px.line(data_frame = df_balance[df_balance['tipo de energía'] == 'Renovable'],
        x = 'Fecha actualización',
        y = 'Valores',
        color = 'nombre',
        color_discrete_sequence = colores_personalizados
)

fig_reno.update_layout(title = 'Evolución de energía diaría renovable')

fig_reno.show()

In [ ]:
fig_no_reno=px.line(data_frame = df_balance[df_balance['tipo de energía'] == 'No-Renovable'],
        x = 'Fecha actualización',
        y = 'Valores',
        color = 'nombre',
        color_discrete_sequence = colores_personalizados
)

fig_no_reno.update_layout(title = 'Evolución de energía diaría no renovable')

fig_no_reno.show()

In [ ]:

fig_dbc = px.line(data_frame = df_balance[df_balance['tipo de energía'] == 'Demanda en b.c.'],
        x = 'Fecha actualización',
        y = 'Valores',
        color = 'nombre',
        color_discrete_sequence = colores_personalizados
)

fig_dbc.update_layout(title = 'Evolución de energía diaría de demanda en barra central')

fig_dbc.show()

In [ ]:
px.scatter(data_frame = df_balance,
            y = 'Valores',
            color = 'nombre',
            color_discrete_sequence = colores_personalizados,
            opacity     = 0.3,
            size        = 'Porcentaje',
            size_max    = 30
          )

In [ ]:
px.scatter(data_frame = df_balance,
            y = 'Valores',
            color = 'tipo de energía',
            color_discrete_sequence = colores_personalizados,
            opacity     = 0.5,
            hover_name  = 'nombre',
            size        = 'Porcentaje',
            size_max    = 30
          )

In [ ]:
px.box(data_frame=df_balance,
       x = 'Valores',
       y = 'nombre',
       color = 'nombre',
       color_discrete_sequence = colores_personalizados)

In [ ]:
px.box(data_frame=df_balance,
       x = 'Valores',
       y = 'tipo de energía',
       color = 'tipo de energía',
       color_discrete_sequence = colores_personalizados)